In [1]:
# !git clone https://github.com/Saiteja-Reddy/Automatic-Text-Scoring.git
import  keras.layers  as  klayers 
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, LSTM, Input, Embedding, GlobalAveragePooling1D, Concatenate, Activation, Lambda, BatchNormalization, Convolution1D, Dropout
from keras.preprocessing.text import Tokenizer
import numpy as np
import nltk
from quadratic_weighted_kappa import QWK
from sklearn.metrics import cohen_kappa_score
from keras.models import Model
from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras import regularizers
from keras import initializers
from scipy import stats

Using TensorFlow backend.


In [2]:
class Neural_Tensor_layer(Layer):
	def __init__(self,output_dim,input_dim=None, **kwargs):
		self.output_dim=output_dim
		self.input_dim=input_dim
		if self.input_dim:
			kwargs['input_shape']=(self.input_dim,)
# 		print("YAYY", input_dim, output_dim)
		super(Neural_Tensor_layer,self).__init__(**kwargs)

	def call(self,inputs,mask=None):
		e1=inputs[0]
		e2=inputs[1]
		batch_size=K.shape(e1)[0]
		k=self.output_dim
		

		feed_forward=K.dot(K.concatenate([e1,e2]),self.V)

		bilinear_tensor_products = [ K.sum((e2 * K.dot(e1, self.W[0])) + self.b, axis=1) ]

		for i in range(k)[1:]:	
			btp=K.sum((e2*K.dot(e1,self.W[i]))+self.b,axis=1)
			bilinear_tensor_products.append(btp)

		result=K.tanh(K.reshape(K.concatenate(bilinear_tensor_products,axis=0),(batch_size,k))+feed_forward)

		return result
    
	def build(self,input_shape):
		mean=0.0
		std=1.0
		k=self.output_dim
		d=self.input_dim
		##truncnorm generate continuous random numbers in given range
		W_val=stats.truncnorm.rvs(-2 * std, 2 * std, loc=mean, scale=std, size=(k,d,d))
		V_val=stats.truncnorm.rvs(-2 * std, 2 * std, loc=mean, scale=std, size=(2*d,k))
		self.W=K.variable(W_val)
		self.V=K.variable(V_val)
		self.b=K.zeros((self.input_dim,))
		self.trainable_weights=[self.W,self.V,self.b]    

	def compute_output_shape(self, input_shape):
		batch_size=input_shape[0][0]
		return(batch_size,self.output_dim)

In [3]:
class Temporal_Mean_Pooling(Layer): # conversion from (samples,timesteps,features) to (samples,features)
	def __init__(self, **kwargs):
		super(Temporal_Mean_Pooling,self).__init__(**kwargs)
		# masked values in x (number_of_samples,time)
		self.supports_masking=True
		# Specifies number of dimensions to each layer
		self.input_spec=InputSpec(ndim=3)
        
	def call(self,x,mask=None):
		if mask is None:
			mask=K.mean(K.ones_like(x),axis=-1)

		mask=K.cast(mask,K.floatx())
				#dimension size single vec/number of samples
		return K.sum(x,axis=-2)/K.sum(mask,axis=-1,keepdims=True)        

	def compute_mask(self,input,mask):
		return None
    
    
	def compute_output_shape(self,input_shape):
		return (input_shape[0],input_shape[2])

In [4]:
EMBEDDING_DIM=300
MAX_NB_WORDS=4000

MAX_SEQUENCE_LENGTH=500
VALIDATION_SPLIT=0.20
DELTA=20

texts=[]
# texts_dev = []
labels=[]
# labels_dev = []
# sentences=[]

originals = []

fp1=open("glove.6B.300d.txt","r", encoding="utf-8")
glove_emb={}
for line in fp1:
	temp=line.split(" ")
	glove_emb[temp[0]]=np.asarray([float(i) for i in temp[1:]])

print("Embedding done")

Embedding done


In [5]:
essay_type = '2'

fp=open("data/training_set_rel3.tsv",'r', encoding="ascii", errors="ignore")
fp.readline()
originals = []
for line in fp:
    temp=line.split("\t")
    if(temp[1]==essay_type): ## why only 4 ?? - evals in prompt specific fashion
        originals.append(float(temp[6]))
# print(originals)
fp.close()
# print(originals)
print("range min - ", min(originals) , " ; range max - ", max(originals))

range_min = min(originals)
range_max = max(originals)

fp=open("data/training_set_rel3.tsv",'r', encoding="ascii", errors="ignore")
fp.readline()
sentences=[]
for line in fp:
    temp=line.split("\t")
    if(temp[1]==essay_type): ## why only 4 ?? - evals in prompt specific fashion
        texts.append(temp[2])
        labels.append((float(temp[6])-range_min)/(range_max-range_min)) ## why ??  - normalize to range [0-1]
        line=temp[2].strip()
        sentences.append(nltk.tokenize.word_tokenize(line))

fp.close()

range min -  1.0  ; range max -  6.0


In [6]:
labels
print(len(labels))

1800


In [7]:
texts[1]

'"Write a persuasive essay to a newspaper reflecting your views on censorship in libraries. Do you believe that certain should be removed i think so be no that yes i think should no person that in chager the book, music, movies, magazines, ect., that be no agure      why do i think if you need that please  think i no thank you please if  i need why do we if know that if i failure the this test i who need to graduate please the children allow to home please yes.          Why do we need to be a prafece person please why do we need to do this why write this assgiment because you mean to be the best teaches ever and ever facebook is my password is @PERSON1  @NUM1 that why i need my myspace is the same thingh but different at same time please know that i need to know i really  i need to my e-mail address is  @EMAIL1 that is my e-mail please work m"'

In [8]:
print("text labels appended %s" %len(texts))

labels=np.asarray(labels)
print(labels)
print(len(labels))

text labels appended 1800
[0.6 0.  0.2 ... 0.2 0.4 0.4]
1800


In [9]:
for i in sentences:
	temp1=np.zeros((1, EMBEDDING_DIM))
	for w in i:
		if(w in glove_emb):
			temp1+=glove_emb[w]
	temp1/=len(i)

In [10]:
tokenizer=Tokenizer(num_words = MAX_NB_WORDS) #num_words=MAX_NB_WORDS) #limits vocabulory size
tokenizer.fit_on_texts(texts) #encoding the text
sequences=tokenizer.texts_to_sequences(texts) #returns list of sequences
word_index=tokenizer.word_index #dictionary mapping, word and specific token for that word...
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH) #padding to max_length


print('Shape of data tensor:', data.shape)

Found 14685 unique tokens.
Shape of data tensor: (1800, 500)


In [11]:
indices=np.arange(data.shape[0]) #with one argument, start=0, step =1
print(data.shape)
np.random.shuffle(indices)
data=data[indices]
print(data.shape)
labels=labels[indices]
# np.reshape(labels, ())
print(labels.shape)
validation_size=int(VALIDATION_SPLIT*data.shape[0])
print(validation_size)

(1800, 500)
(1800, 500)
(1800,)
360


In [12]:
x_train=data[:-validation_size] #data-validation data
print(x_train.shape)
# print(x_train)
# print(labels)
y_train=labels[:-validation_size]
# print(y_train.transpose)
print(y_train.shape)
# y_train = np.reshape(y_train, (1427, 1))
# print(y_train_new)
# print(y_train)
x_val=data[-validation_size:]
print(x_val.shape)
y_val=labels[-validation_size:]

(1440, 500)
(1440,)
(360, 500)


In [13]:
embedding_matrix = np.zeros((len(word_index), EMBEDDING_DIM))
print(embedding_matrix.shape)

(14685, 300)


In [14]:
for word,i in word_index.items():
	if(i>=len(word_index)):
		continue
	if word in glove_emb:
			embedding_matrix[i]=glove_emb[word]
vocab_size=len(word_index)
print(vocab_size)

14685


In [15]:
embedding_layer=Embedding(vocab_size,EMBEDDING_DIM,weights=[embedding_matrix],
							input_length=MAX_SEQUENCE_LENGTH,
							mask_zero=True,
							trainable=False)
# print(embedding_layer.shape)
side_embedding_layer=Embedding(vocab_size,EMBEDDING_DIM,weights=[embedding_matrix],
							input_length=MAX_SEQUENCE_LENGTH,
							mask_zero=False,
							trainable=False)

W0219 16:21:36.109570 140247617083200 deprecation_wrapper.py:119] From /home/rajivratn/anaconda3/envs/dependency/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [16]:
def SKIPFLOW(lstm_dim=50, lr=1e-4, lr_decay=1e-6, k=4, eta=3, delta=50, activation="relu", maxlen=MAX_SEQUENCE_LENGTH, seed=None):
    e = Input(name='essay',shape=(maxlen,))
    print("e", e)
#     trad_feats=Input(shape=(7,))
#     print("trad_feats", trad_feats)
    embed = embedding_layer(e)
    print(embed.shape)
    lstm_layer=LSTM(lstm_dim,return_sequences=True)
    print(lstm_layer)
    hidden_states=lstm_layer(embed)
    htm=Temporal_Mean_Pooling()(hidden_states)    
    side_embed = side_embedding_layer(e)
    side_hidden_states=lstm_layer(side_embed)    
    tensor_layer=Neural_Tensor_layer(output_dim=k,input_dim=500)
#     print(input_dim, output_dim)
    pairs = [((eta + i * delta) % maxlen, (eta + i * delta + delta) % maxlen) for i in range(maxlen // delta)]
    hidden_pairs = [ (Lambda(lambda t: t[:, p[0], :])(side_hidden_states), Lambda(lambda t: t[:, p[1], :])(side_hidden_states)) for p in pairs]
    sigmoid = Dense(1, activation="sigmoid", kernel_initializer=initializers.glorot_normal(seed=seed))
    coherence = [sigmoid(tensor_layer([hp[0], hp[1]])) for hp in hidden_pairs]
    co_tm=Concatenate()(coherence[:]+[htm])
    dense = Dense(256, activation=activation,kernel_initializer=initializers.glorot_normal(seed=seed))(co_tm)
    dense = Dense(128, activation=activation,kernel_initializer=initializers.glorot_normal(seed=seed))(dense)
    dense = Dense(64, activation=activation,kernel_initializer=initializers.glorot_normal(seed=seed))(dense)
    out = Dense(1, activation="sigmoid")(dense)
    model = Model(inputs=[e], outputs=[out])
    print("input", [e])
    print("outputs", out)
    adam = Adam(lr=lr, decay=lr_decay)
    model.compile(loss="mean_squared_error", optimizer=adam, metrics=["MSE"])
    return model

In [17]:
# from keras.utils.vis_utils import plot_model
earlystopping = EarlyStopping(monitor="val_mean_squared_error", patience=5)
sf_2_500 = SKIPFLOW(lstm_dim=500, lr=2e-4, lr_decay=2e-6, k=4, eta=13, delta=50, activation="relu", seed=None)
sf_2_500.summary()
# plot_model(sf_1)

W0219 16:22:04.003850 140247617083200 deprecation_wrapper.py:119] From /home/rajivratn/anaconda3/envs/dependency/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0219 16:22:04.009980 140247617083200 deprecation_wrapper.py:119] From /home/rajivratn/anaconda3/envs/dependency/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0219 16:22:04.025536 140247617083200 deprecation_wrapper.py:119] From /home/rajivratn/anaconda3/envs/dependency/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0219 16:22:04.026233 140247617083200 deprecation_wrapper.py:119] From /home/rajivratn/anaconda3/envs/dependency/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:181: The 

e Tensor("essay:0", shape=(?, 500), dtype=float32)
(?, 500, 300)


W0219 16:22:05.823732 140247617083200 deprecation.py:323] From /home/rajivratn/anaconda3/envs/dependency/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:2974: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0219 16:22:07.050283 140247617083200 deprecation_wrapper.py:119] From /home/rajivratn/anaconda3/envs/dependency/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



input [<tf.Tensor 'essay:0' shape=(?, 500) dtype=float32>]
outputs Tensor("dense_5/Sigmoid:0", shape=(?, 1), dtype=float32)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
essay (InputLayer)              (None, 500)          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 500, 300)     4405500     essay[0][0]                      
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 500, 500)     1602000     embedding_1[0][0]                
                                                                 embedding_2[0][0]                
____________________________________________________________________________________

In [18]:
# print(sf_1)
epochs = 100
# epochs = 1000
print(type(x_train))
# y_train = np.asarray(y_train)
print(type(y_train))

hist = sf_2_500.fit(x_train, y_train, batch_size=32, epochs=epochs, validation_data=([x_val], y_val), callbacks=[earlystopping])

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
Train on 1440 samples, validate on 360 samples
Epoch 1/100


ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted: OOM when allocating tensor with shape[500,32,300] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node lstm_1_1/transpose}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[loss/mul/_469]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted: OOM when allocating tensor with shape[500,32,300] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node lstm_1_1/transpose}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

0 successful operations.
0 derived errors ignored.

In [ ]:
y_pred=sf_2_500.predict([x_val])
y_pred

In [23]:
y_val_fin = [int(round(a*(range_max-range_min)+range_min)) for a in y_val]
print(y_val_fin)

[10, 8, 9, 8, 6, 9, 10, 9, 10, 10, 9, 10, 8, 8, 8, 9, 8, 9, 10, 8, 8, 8, 7, 8, 8, 8, 8, 10, 8, 8, 9, 12, 8, 8, 10, 8, 9, 9, 10, 11, 8, 8, 11, 10, 8, 9, 8, 8, 9, 10, 9, 12, 7, 10, 8, 8, 12, 11, 8, 10, 8, 9, 7, 9, 10, 7, 11, 6, 9, 7, 11, 8, 9, 8, 7, 11, 8, 8, 4, 6, 10, 8, 10, 9, 9, 8, 8, 7, 10, 8, 9, 8, 6, 9, 10, 10, 8, 12, 8, 10, 8, 10, 8, 11, 7, 8, 10, 7, 8, 10, 10, 10, 8, 8, 9, 8, 8, 9, 10, 9, 8, 8, 8, 6, 6, 6, 8, 8, 8, 9, 8, 8, 7, 8, 8, 6, 10, 8, 9, 6, 9, 12, 9, 8, 10, 9, 9, 7, 10, 7, 6, 8, 9, 10, 7, 10, 7, 6, 8, 10, 9, 8, 8, 6, 8, 9, 10, 5, 9, 10, 8, 12, 5, 6, 8, 8, 11, 10, 8, 6, 9, 7, 9, 8, 11, 8, 10, 10, 8, 10, 4, 9, 8, 8, 9, 10, 8, 8, 8, 8, 8, 8, 9, 8, 8, 7, 9, 8, 9, 8, 8, 8, 8, 11, 9, 10, 7, 2, 10, 8, 10, 8, 9, 8, 9, 8, 10, 10, 8, 8, 10, 7, 8, 8, 8, 10, 10, 8, 8, 7, 10, 9, 9, 11, 8, 10, 8, 8, 8, 10, 12, 6, 9, 8, 10, 6, 8, 12, 4, 11, 8, 8, 8, 11, 6, 10, 9, 8, 7, 8, 8, 11, 9, 8, 10, 8, 11, 7, 8, 8, 12, 8, 8, 6, 9, 10, 9, 9, 8, 8, 6, 7, 8, 6, 5, 8, 10, 10, 8, 8, 8, 6, 10, 10, 10, 8

In [25]:
y_pred_fin =[int(round(a*(range_max-range_min)+range_min)) for a in y_pred.reshape(356).tolist()]
print(y_pred_fin)

[9, 8, 8, 10, 7, 9, 11, 9, 10, 8, 9, 10, 8, 9, 8, 9, 8, 9, 9, 8, 8, 8, 7, 9, 8, 8, 8, 10, 8, 8, 9, 10, 8, 8, 9, 8, 9, 9, 10, 11, 8, 7, 10, 9, 9, 8, 8, 8, 10, 8, 9, 10, 7, 9, 9, 9, 10, 10, 8, 9, 10, 7, 7, 9, 10, 7, 9, 5, 10, 7, 9, 8, 9, 10, 8, 10, 8, 9, 5, 6, 9, 7, 10, 10, 8, 6, 9, 8, 9, 8, 8, 9, 7, 8, 9, 9, 9, 11, 9, 9, 8, 10, 9, 10, 9, 9, 10, 7, 9, 10, 10, 10, 9, 8, 9, 8, 8, 9, 10, 8, 8, 8, 8, 6, 8, 7, 7, 8, 8, 9, 8, 8, 7, 9, 9, 5, 10, 8, 10, 7, 8, 10, 8, 9, 10, 8, 10, 9, 10, 7, 7, 9, 9, 8, 8, 11, 7, 7, 9, 10, 10, 9, 8, 5, 8, 10, 10, 5, 9, 10, 8, 10, 6, 7, 8, 8, 10, 10, 9, 8, 9, 7, 8, 8, 10, 8, 10, 10, 9, 10, 6, 9, 8, 7, 10, 10, 8, 8, 10, 8, 8, 9, 9, 8, 8, 8, 8, 8, 9, 9, 8, 9, 10, 10, 9, 9, 8, 5, 9, 8, 10, 9, 8, 10, 9, 9, 10, 10, 9, 8, 10, 8, 8, 8, 9, 10, 10, 9, 8, 7, 10, 9, 10, 10, 9, 10, 9, 8, 8, 8, 11, 8, 8, 8, 10, 7, 9, 11, 5, 11, 9, 8, 8, 10, 6, 9, 8, 9, 6, 10, 8, 10, 9, 9, 10, 8, 9, 8, 9, 8, 10, 9, 9, 7, 9, 9, 9, 9, 8, 8, 5, 7, 8, 6, 5, 9, 9, 9, 8, 8, 8, 7, 9, 9, 10, 8, 9, 9, 8,

In [26]:
print(cohen_kappa_score(y_val_fin,y_pred_fin,weights="quadratic"))

0.7962121212121211


In [29]:
def Cmatrix(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Returns the confusion matrix between rater's ratings
    """
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(rater_a + rater_b)
    if max_rating is None:
        max_rating = max(rater_a + rater_b)
    num_ratings = int(max_rating - min_rating + 1)
    conf_mat = [[0 for i in range(num_ratings)]
                for j in range(num_ratings)]
    for a, b in zip(rater_a, rater_b):
        conf_mat[a - min_rating][b - min_rating] += 1
    return conf_mat


def histogram(ratings, min_rating=None, max_rating=None):
    """
    Returns the counts of each type of rating that a rater made
    """
    if min_rating is None:
        min_rating = min(ratings)
    if max_rating is None:
        max_rating = max(ratings)
    num_ratings = int(max_rating - min_rating + 1)
    hist_ratings = [0 for x in range(num_ratings)]
    for r in ratings:
        hist_ratings[r - min_rating] += 1
    return hist_ratings


def QWK_new(y, y_pred):
    """
    Calculates the quadratic weighted kappa
    axquadratic_weighted_kappa calculates the quadratic weighted kappa
    value, which is a measure of inter-rater agreement between two raters
    that provide discrete numeric ratings.  Potential values range from -1
    (representing complete disagreement) to 1 (representing complete
    agreement).  A kappa value of 0 is expected if all agreement is due to
    chance.
    quadratic_weighted_kappa(rater_a, rater_b), where rater_a and rater_b
    each correspond to a list of integer ratings.  These lists must have the
    same length.
    The ratings should be integers, and it is assumed that they contain
    the complete range of possible ratings.
    quadratic_weighted_kappa(X, min_rating, max_rating), where min_rating
    is the minimum possible rating, and max_rating is the maximum possible
    rating
    """
    rater_a = y
    rater_b = y_pred
    min_rating=None
    max_rating=None
    rater_a = np.array(rater_a, dtype=int)
    rater_b = np.array(rater_b, dtype=int)
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(min(rater_a), min(rater_b))
    if max_rating is None:
        max_rating = max(max(rater_a), max(rater_b))
    conf_mat = Cmatrix(rater_a, rater_b,
                                min_rating, max_rating)
    num_ratings = len(conf_mat)
    num_scored_items = float(len(rater_a))

    hist_rater_a = histogram(rater_a, min_rating, max_rating)
    hist_rater_b = histogram(rater_b, min_rating, max_rating)

    numerator = 0.0
    denominator = 0.0

    for i in range(num_ratings):
        for j in range(num_ratings):
            expected_count = (hist_rater_a[i] * hist_rater_b[j]
                              / num_scored_items)
            d = pow(i - j, 2.0) / pow(num_ratings - 1, 2.0)
            numerator += d * conf_mat[i][j] / num_scored_items
            denominator += d * expected_count / num_scored_items

    return (1.0 - numerator / denominator)

In [30]:
QWK_new(y_val_fin, y_pred_fin)

0.7943090425240287

In [32]:
sf_1.save('model_final/1_model.h5')

In [33]:
y_pred*(range_max-range_min)+range_min

array([[ 9.077402 ],
       [ 7.9122553],
       [ 7.7343826],
       [ 9.691392 ],
       [ 7.1335278],
       [ 8.686952 ],
       [10.531337 ],
       [ 9.339651 ],
       [ 9.860868 ],
       [ 8.380489 ],
       [ 9.469806 ],
       [ 9.731863 ],
       [ 8.236764 ],
       [ 8.833696 ],
       [ 8.161226 ],
       [ 9.201469 ],
       [ 8.39722  ],
       [ 8.848019 ],
       [ 9.321454 ],
       [ 8.1174755],
       [ 8.32741  ],
       [ 7.6823816],
       [ 7.354194 ],
       [ 8.8032055],
       [ 7.743875 ],
       [ 8.066136 ],
       [ 7.7435837],
       [10.297057 ],
       [ 7.5326347],
       [ 8.176024 ],
       [ 9.4363   ],
       [10.069847 ],
       [ 8.208425 ],
       [ 8.4025345],
       [ 9.248013 ],
       [ 8.388559 ],
       [ 9.170399 ],
       [ 8.650791 ],
       [ 9.601019 ],
       [10.586998 ],
       [ 8.031967 ],
       [ 7.1991963],
       [10.093216 ],
       [ 8.512126 ],
       [ 9.358947 ],
       [ 7.9787297],
       [ 7.933788 ],
       [ 8.26

In [36]:
y_pred_fin

[9,
 8,
 8,
 10,
 7,
 9,
 11,
 9,
 10,
 8,
 9,
 10,
 8,
 9,
 8,
 9,
 8,
 9,
 9,
 8,
 8,
 8,
 7,
 9,
 8,
 8,
 8,
 10,
 8,
 8,
 9,
 10,
 8,
 8,
 9,
 8,
 9,
 9,
 10,
 11,
 8,
 7,
 10,
 9,
 9,
 8,
 8,
 8,
 10,
 8,
 9,
 10,
 7,
 9,
 9,
 9,
 10,
 10,
 8,
 9,
 10,
 7,
 7,
 9,
 10,
 7,
 9,
 5,
 10,
 7,
 9,
 8,
 9,
 10,
 8,
 10,
 8,
 9,
 5,
 6,
 9,
 7,
 10,
 10,
 8,
 6,
 9,
 8,
 9,
 8,
 8,
 9,
 7,
 8,
 9,
 9,
 9,
 11,
 9,
 9,
 8,
 10,
 9,
 10,
 9,
 9,
 10,
 7,
 9,
 10,
 10,
 10,
 9,
 8,
 9,
 8,
 8,
 9,
 10,
 8,
 8,
 8,
 8,
 6,
 8,
 7,
 7,
 8,
 8,
 9,
 8,
 8,
 7,
 9,
 9,
 5,
 10,
 8,
 10,
 7,
 8,
 10,
 8,
 9,
 10,
 8,
 10,
 9,
 10,
 7,
 7,
 9,
 9,
 8,
 8,
 11,
 7,
 7,
 9,
 10,
 10,
 9,
 8,
 5,
 8,
 10,
 10,
 5,
 9,
 10,
 8,
 10,
 6,
 7,
 8,
 8,
 10,
 10,
 9,
 8,
 9,
 7,
 8,
 8,
 10,
 8,
 10,
 10,
 9,
 10,
 6,
 9,
 8,
 7,
 10,
 10,
 8,
 8,
 10,
 8,
 8,
 9,
 9,
 8,
 8,
 8,
 8,
 8,
 9,
 9,
 8,
 9,
 10,
 10,
 9,
 9,
 8,
 5,
 9,
 8,
 10,
 9,
 8,
 10,
 9,
 9,
 10,
 10,
 9,
 8,
 10,
 8,
 8,
 8,
 9,
 10

In [37]:
y_val_fin

[10,
 8,
 9,
 8,
 6,
 9,
 10,
 9,
 10,
 10,
 9,
 10,
 8,
 8,
 8,
 9,
 8,
 9,
 10,
 8,
 8,
 8,
 7,
 8,
 8,
 8,
 8,
 10,
 8,
 8,
 9,
 12,
 8,
 8,
 10,
 8,
 9,
 9,
 10,
 11,
 8,
 8,
 11,
 10,
 8,
 9,
 8,
 8,
 9,
 10,
 9,
 12,
 7,
 10,
 8,
 8,
 12,
 11,
 8,
 10,
 8,
 9,
 7,
 9,
 10,
 7,
 11,
 6,
 9,
 7,
 11,
 8,
 9,
 8,
 7,
 11,
 8,
 8,
 4,
 6,
 10,
 8,
 10,
 9,
 9,
 8,
 8,
 7,
 10,
 8,
 9,
 8,
 6,
 9,
 10,
 10,
 8,
 12,
 8,
 10,
 8,
 10,
 8,
 11,
 7,
 8,
 10,
 7,
 8,
 10,
 10,
 10,
 8,
 8,
 9,
 8,
 8,
 9,
 10,
 9,
 8,
 8,
 8,
 6,
 6,
 6,
 8,
 8,
 8,
 9,
 8,
 8,
 7,
 8,
 8,
 6,
 10,
 8,
 9,
 6,
 9,
 12,
 9,
 8,
 10,
 9,
 9,
 7,
 10,
 7,
 6,
 8,
 9,
 10,
 7,
 10,
 7,
 6,
 8,
 10,
 9,
 8,
 8,
 6,
 8,
 9,
 10,
 5,
 9,
 10,
 8,
 12,
 5,
 6,
 8,
 8,
 11,
 10,
 8,
 6,
 9,
 7,
 9,
 8,
 11,
 8,
 10,
 10,
 8,
 10,
 4,
 9,
 8,
 8,
 9,
 10,
 8,
 8,
 8,
 8,
 8,
 8,
 9,
 8,
 8,
 7,
 9,
 8,
 9,
 8,
 8,
 8,
 8,
 11,
 9,
 10,
 7,
 2,
 10,
 8,
 10,
 8,
 9,
 8,
 9,
 8,
 10,
 10,
 8,
 8,
 10,
 7,
 8,
 8,
 8,

In [38]:
sf_1.save_weights('weights_final/1_weights.h5')

In [41]:
!pip install pydot
# !pip install graphviz
from keras.utils import plot_model
plot_model(sf_1)

  Using cached https://files.pythonhosted.org/packages/33/d1/b1479a770f66d962f545c2101630ce1d5592d90cb4f083d38862e93d16d2/pydot-1.4.1-py2.py3-none-any.whl


ImportError: Failed to import `pydot`. Please install `pydot`. For example with `pip install pydot`.